# glm-met

Example use of the glm-met Python package to download meteorological data and format it for use with the General Lake Model (GLM).

In [ ]:
# install glm-met - comment if this step is not required
# !pip install glm-met
!pip install glm-met

## Historical data

Here, we'll be working with the NASA POWER API which lets you download hourly solar and meteorological data.

First, we need to create a `glm_met.nasa_power.nasa_power.Power` object, which stores API query attributes and has methods for downloading and processing the data.

In [2]:
import pprint
import pandas as pd
import os
import glm_met.nasa_power.nasa_power as nasa_power

# initialise a Power object
# the object attributes are set to:
# - query the NASA POWER API 
# - query for hourly met data from 2020 to 2022
# - query a location in Western Australia

power = nasa_power.Power(
    location=(116.6, -32.17), 
    date_range=("20200101", "20221231"), 
    met_data=None,
    glm_met_data=None,
    parameters=None,
)

Once we have created the `Power` object, `power`, we can call its `get_variables()` method to make a request to the NASA POWER API and download the data. The downloaded data will be stored in `powers`'s `met_data` attribute. The `met_data` attribute's type is of `MetData` which has two attributes. `power.met_data.metadata` is a dict of metadata about the request to the NASA POWER API and `power.met_data.data` is a DataFrame object of meteorological data.

In [3]:
power.get_variables(request_settings=None)

In [ ]:
# view the metadata
pprint.pprint(power.met_data.metadata)

In [ ]:
# view the data
power.met_data.data.head()

The GLM model requires meteorological data with a specific format and units. If the required meteorological variables to run GLM have been requested from NASA POWER, `power`'s `convert_to_glm()` method can be used reformat the data. The GLM formatted data is a DataFrame that is stored in `power`'s `glm_met_data` attribute.

In [ ]:
# convert data to GLM format
power.convert_to_glm()

# visualise GLM formatted data
power.glm_met_data.head()

Finally, we can save the GLM formatted meteorological data to a csv file for use with the GLM model.

In [ ]:
power.write_glm_met(path=os.getcwd(), zip_f=False, fname="met.csv")

if "met.csv" in os.listdir(os.getcwd()):
    print("met.csv downloaded OK")